# Wine Quality Prediction using SVM and Grid Search
This notebook explores the performance of an **SVM (Support Vector Machine)** model on the Wine Quality dataset.
We will:
- Train an initial SVM model with default parameters.
- Use **Grid Search** to find the best hyperparameters.
- Evaluate the model before and after tuning.


In [ ]:
!pip install scikit-learn

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler , QuantileTransformer
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

## Loading the Dataset
We use the **Wine Quality dataset**, which contains physicochemical properties of wines along with their quality scores.
Let's load the dataset and inspect its structure.


In [9]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
df = pd.read_csv(url, sep=';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [11]:
df.isnull().sum()
df.info()
# as we can see there are no null values in the dataset so no need for cleaning

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [14]:
# Split into features and target
X = df.drop(columns=['quality'])
y = df['quality']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Training a Baseline SVM Model
We will first train an SVM classifier with default parameters to establish a baseline performance.


In [17]:
#define pipeline we here define two parameters in pipeline scaler and the model
pipe=Pipeline([('scaler' , StandardScaler()) , ('model' , SVC())])
#fitting the pipe
pipe.fit(X_train , y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', SVC())])

In [18]:
y_pred_pipe = pipe.predict(X_test)

## Model Evaluation (Before Hyperparameter Tuning)
We evaluate the model using classification metrics such as **accuracy, precision, recall, and F1-score**.


In [20]:
print("Before Hypertuning")
print(classification_report(y_test , y_pred_pipe))

Before Hypertuning
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        10
           5       0.65      0.76      0.70       130
           6       0.56      0.64      0.60       132
           7       0.56      0.21      0.31        42
           8       0.00      0.00      0.00         5

    accuracy                           0.60       320
   macro avg       0.30      0.27      0.27       320
weighted avg       0.57      0.60      0.57       320



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [28]:
pipe.get_params()
# these are the parameters of the pipeline which we are going to chane in grid-search

{'memory': None,
 'steps': [('scaler', StandardScaler()), ('model', SVC())],
 'transform_input': None,
 'verbose': False,
 'scaler': StandardScaler(),
 'model': SVC(),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'model__C': 1.0,
 'model__break_ties': False,
 'model__cache_size': 200,
 'model__class_weight': None,
 'model__coef0': 0.0,
 'model__decision_function_shape': 'ovr',
 'model__degree': 3,
 'model__gamma': 'scale',
 'model__kernel': 'rbf',
 'model__max_iter': -1,
 'model__probability': False,
 'model__random_state': None,
 'model__shrinking': True,
 'model__tol': 0.001,
 'model__verbose': False}

## Hyperparameter Tuning with Grid Search
To improve model performance, we use **GridSearchCV** to tune hyperparameters such as:
- **C (Regularization Parameter)**
- **Kernel (linear, rbf, poly)**
- **Gamma (scale, auto)**

This will help us find the best combination of hyperparameters.


In [29]:
# implementing grid search
params = {
    'model__C': [0.1, 1, 10],
    'model__kernel': ['linear', 'rbf', 'poly'],
    'model__gamma': ['scale', 'auto']
}
grid_search=GridSearchCV(
    estimator=pipe,
    param_grid=params,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy'

)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', SVC())]),
             n_jobs=-1,
             param_grid={'model__C': [0.1, 1, 10],
                         'model__gamma': ['scale', 'auto'],
                         'model__kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy', verbose=2)

In [34]:
results_df = pd.DataFrame(grid_search.cv_results_)
sorted_results=results_df.sort_values(by='rank_test_score')
sorted_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__C,param_model__gamma,param_model__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
13,0.126604,0.005490,0.031513,0.000452,10.0,scale,rbf,"{'model__C': 10, 'model__gamma': 'scale', 'mod...",0.636719,0.609375,0.593750,0.656250,0.686275,0.636474,0.032957,1
16,0.224434,0.018703,0.053982,0.015149,10.0,auto,rbf,"{'model__C': 10, 'model__gamma': 'auto', 'mode...",0.636719,0.609375,0.593750,0.656250,0.686275,0.636474,0.032957,1
10,0.100539,0.002078,0.032465,0.000933,1.0,auto,rbf,"{'model__C': 1, 'model__gamma': 'auto', 'model...",0.613281,0.574219,0.597656,0.640625,0.690196,0.623195,0.039855,3
7,0.099053,0.001437,0.032149,0.000545,1.0,scale,rbf,"{'model__C': 1, 'model__gamma': 'scale', 'mode...",0.613281,0.574219,0.597656,0.640625,0.690196,0.623195,0.039855,3
17,0.257143,0.022184,0.028567,0.009851,10.0,auto,poly,"{'model__C': 10, 'model__gamma': 'auto', 'mode...",0.601562,0.578125,0.605469,0.613281,0.635294,0.606746,0.018468,5
14,0.168381,0.026712,0.020744,0.004882,10.0,scale,poly,"{'model__C': 10, 'model__gamma': 'scale', 'mod...",0.601562,0.578125,0.605469,0.613281,0.635294,0.606746,0.018468,5
11,0.105035,0.006862,0.020612,0.001820,1.0,auto,poly,"{'model__C': 1, 'model__gamma': 'auto', 'model...",0.554688,0.585938,0.597656,0.613281,0.654902,0.601293,0.032981,7
8,0.108177,0.009419,0.019216,0.000252,1.0,scale,poly,"{'model__C': 1, 'model__gamma': 'scale', 'mode...",0.554688,0.585938,0.597656,0.613281,0.654902,0.601293,0.032981,7
4,0.103132,0.003354,0.034478,0.000493,0.1,auto,rbf,"{'model__C': 0.1, 'model__gamma': 'auto', 'mod...",0.558594,0.570312,0.589844,0.601562,0.643137,0.592690,0.029309,9
1,0.170508,0.041268,0.054146,0.013169,0.1,scale,rbf,"{'model__C': 0.1, 'model__gamma': 'scale', 'mo...",0.558594,0.570312,0.589844,0.601562,0.643137,0.592690,0.029309,9


In [35]:
print("Best Parameters:", grid_search.best_params_)
# also we can see from above dataframe that this model ranks as number 1 according to test scores

Best Parameters: {'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'rbf'}


## Training the SVM Model with Best Parameters
Now, we train an SVM model using the optimal hyperparameters obtained from Grid Search.


In [36]:
best_svm = grid_search.best_estimator_
y_pred_best = best_svm.predict(X_test)


In [38]:
print("Tuned SVM Performance:")
print(classification_report(y_test, y_pred_best))

Tuned SVM Performance:
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        10
           5       0.67      0.75      0.71       130
           6       0.59      0.61      0.60       132
           7       0.55      0.40      0.47        42
           8       0.00      0.00      0.00         5

    accuracy                           0.61       320
   macro avg       0.30      0.29      0.30       320
weighted avg       0.59      0.61      0.60       320



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# **Comparison of SVM Performance Before and After Hyperparameter Tuning**

## **1. Accuracy Improvement**
- Before tuning: **0.60**
- After tuning: **0.61**
- **Slight improvement** in overall accuracy.

## **2. Macro Average Scores**
| Metric       | Before Tuning | After Tuning | Change |
|-------------|--------------|--------------|--------|
| Precision   | 0.30         | 0.30         | No change |
| Recall      | 0.27         | 0.29         | **+0.02** |
| F1-score    | 0.27         | 0.30         | **+0.03** |

- **Recall and F1-score improved slightly after tuning.**
- Precision remained the same.

## **3. Weighted Average Scores**
| Metric       | Before Tuning | After Tuning | Change |
|-------------|--------------|--------------|--------|
| Precision   | 0.57         | 0.59         | **+0.02** |
| Recall      | 0.60         | 0.61         | **+0.01** |
| F1-score    | 0.57         | 0.60         | **+0.03** |

- **Slight improvements in all weighted metrics.**
- **Better handling of class imbalance after tuning.**

## **4. Class-wise Performance**
| Class | Precision (Before) | Precision (After) | Recall (Before) | Recall (After) | F1-score (Before) | F1-score (After) |
|-------|-------------------|-------------------|-----------------|---------------|-----------------|-----------------|
| 3     | 0.00              | 0.00              | 0.00            | 0.00          | 0.00            | 0.00            |
| 4     | 0.00              | 0.00              | 0.00            | 0.00          | 0.00            | 0.00            |
| 5     | **0.65**          | **0.67**          | 0.76            | **0.75**      | 0.70            | **0.71**        |
| 6     | 0.56              | **0.59**          | **0.64**        | **0.61**      | 0.60            | 0.60            |
| 7     | **0.56**          | 0.55              | **0.21**        | **0.40**      | **0.31**        | **0.47**        |
| 8     | 0.00              | 0.00              | 0.00            | 0.00          | 0.00            | 0.00            |

### **Key Observations**
- **Class 5 & 6** improved in **precision** and retained good recall.
- **Class 7 recall improved significantly** from **0.21 → 0.40**, boosting its F1-score.
- **Classes 3, 4, and 8 still have poor performance**, likely due to **data imbalance**.

## **Conclusion**
- Hyperparameter tuning resulted in **marginal improvements** in overall accuracy, recall, and F1-score.
- Improvements were **more significant for minority classes**, particularly Class 7.
- Further improvements could be achieved by **handling class imbalance (e.g., SMOTE, class weighting)**.

